## Sistema Multi-Agente per lo Sviluppo Software con Haystack

Questo notebook implementa un **sistema multi-agente** bassu su **Haystack** e integrato con **OpenRouter.AI**, in cui più **modelli linguistici di grandi dimensioni (LLM)** collaborano in modo coordinato, ciascuno con un **ruolo specifico** all'interno del ciclo di vita di un progetto software.  
L'obiettivo è simulare un team di sviluppo virtuale in grado di gestire un progetto software dall'analisi iniziale fino alla verifica finale.

---

### 👥 Ruoli e Responsabilità

- **Project Manager**
- **Solution Architect**
- **Technical Lead**
- **Frontend Developer**
- **Backend Developer**
- **Database Administrator**
- **QA & Test Engineer**

---

### 🔁 Flusso del Processo Collaborativo

```text
Project Manager → Solution Architect → Technical Lead
                              ↓
       Frontend Dev ←→ Backend Dev ←→ DB Admin
                              ↓
                     QA & Test Engineer
```

---

### 🧭 Descrizione del Processo

- Il **Project Manager** riceve in input i parametri utente (`project_name`, `project_type`, `project_requirements`) e redige un documento di *Specifiche di Progetto*.
- Il **Solution Architect** definisce l'architettura tecnica del sistema, selezionando i componenti principali e le linee guida progettuali.
- Il **Technical Lead** costruisce un *piano di implementazione* articolato in un backlog di attività, assegnando priorità, responsabili e dipendenze.
- Gli sviluppatori **Frontend**, **Backend** e il **Database Administrator** realizzano le varie componenti applicative seguendo il piano stabilito.
- Il **QA & Test Engineer** verifica la correttezza, la robustezza e la coerenza delle implementazioni rispetto ai requisiti iniziali e alle specifiche tecniche.

I deliverables prodotti da ogni agent vengono salvati in formato MarkDown nella cartella /outputs

In [ ]:
import os
import yaml
from dotenv import load_dotenv
from haystack import Pipeline
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders import PromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret
from typing import List, Dict, Any

# Caricamento variabili d'ambiente
load_dotenv("vars.env")

In [ ]:
# Parametri del progetto
project_name = "ToDo list per la gestione di attività personali"
project_type = "Web App"
project_requirements = [
    # Requisiti Funzionali
    "L'utente può creare nuove attività con titolo, descrizione, data di scadenza e priorità",
    "Le attività devono essere visualizzabili in una lista ordinabile per data, priorità o stato",
    "L'utente può modificare attività esistenti (titolo, descrizione, scadenza, priorità)",
    "L'utente può eliminare attività dalla lista",
    "L'utente può contrassegnare un'attività come completata",
    "È possibile filtrare le attività per stato (completate/in sospeso), priorità e cercarle per testo",
    "Il sistema può inviare notifiche per attività prossime alla scadenza",
    "Le attività possono essere assegnate a categorie personalizzate o etichette (tag)",
    "Il software consente di salvare e ripristinare le attività (backup locale o cloud)",
    "Le attività si sincronizzano tra più dispositivi con lo stesso account",
   
    # Requisiti Non Funzionali
    "Interfaccia semplice, intuitiva e usabile anche da utenti non esperti",
    "Tempi di risposta rapidi anche con molte attività",
    "Supporto per più piattaforme (desktop, mobile, web)",
    "Il sistema garantisce l'integrità e la persistenza dei dati inseriti",
    "I dati dell'utente devono essere protetti, soprattutto se salvati nel cloud",
    "Il software deve essere scalabile per l'aggiunta futura di nuove funzionalità",

    # Requisiti Tecnici
    "Supporto per frontend in React, Angular, Vue o Flutter",
    "Utilizzo di backend in Node.js, Python (Django/Flask) o Java (Spring Boot)",
    "Persistenza dati locale (SQLite) o remota tramite API REST e database relazionale",
    "Architettura del software basata su MVC o MVVM",
    "Integrazione con notifiche push (Firebase, OneSignal) e autenticazione (OAuth, Google Sign-In)",
    "Presenza di test unitari e di integrazione per garantire la qualità del software"       
]

In [ ]:
class HaystackAgent:
    """Classe per rappresentare un agente nel sistema Haystack"""
    
    def __init__(self, role: str, goal: str, backstory: str, llm_config: dict = None, verbose: bool = False):
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.verbose = verbose
        
        # Configurazione di default per llm_config
        default_llm_config = {
            "model": "openai/gpt-3.5-turbo",
            "temperature": 0.7
        }
        
        self.llm_config = llm_config or default_llm_config
        
          # Creazione del generatore OpenAI con Secret per l'API key
        api_key_value = os.getenv("OPENAI_API_KEY")
        if not api_key_value:
            raise ValueError("OPENAI_API_KEY non trovata nelle variabili d'ambiente")
        
        self.generator = OpenAIGenerator(
            api_key=Secret.from_token(api_key_value),  # Usa Secret.from_token() invece di passare direttamente la stringa
            api_base_url=os.getenv("OPENAI_API_BASE"),
            model=self.llm_config["model"],
            generation_kwargs={
                "temperature": self.llm_config["temperature"]
            }
        )
        
        # Template del prompt per l'agente
        self.prompt_template = """
        Ruolo: {{ role }}
        Obiettivo: {{ goal }}
        Contesto: {{ backstory }}
        
        Task da eseguire: {{ task_description }}
        
        {% if context_data %}
        Informazioni di contesto dai task precedenti:
        {{ context_data }}
        {% endif %}
        
        {% if expected_output %}
        Output atteso: {{ expected_output }}
        {% endif %}
        
        {% if project_name %}
        Nome del progetto: {{ project_name }}
        {% endif %}
        
        {% if project_type %}
        Tipo di progetto: {{ project_type }}
        {% endif %}
        
        {% if project_requirements %}
        Requisiti del progetto:
        {% for req in project_requirements %}
        - {{ req }}
        {% endfor %}
        {% endif %}
        
        Fornisci una risposta dettagliata e professionale secondo il tuo ruolo.
        """
        
        # Creazione della pipeline
        self.pipeline = Pipeline()
        prompt_builder = PromptBuilder(template=self.prompt_template)
        
        self.pipeline.add_component("prompt_builder", prompt_builder)
        self.pipeline.add_component("generator", self.generator)
        
        self.pipeline.connect("prompt_builder", "generator")
    
    def execute_task(self, task_description: str, expected_output: str = "", 
                    context_data: str = "", **kwargs) -> str:
        """Esegue un task utilizzando l'agente"""
        
        if self.verbose:
            print(f"[{self.role}] Eseguendo task: {task_description[:100]}...")
        
        result = self.pipeline.run({
            "prompt_builder": {
                "role": self.role,
                "goal": self.goal,
                "backstory": self.backstory,
                "task_description": task_description,
                "expected_output": expected_output,
                "context_data": context_data,
                **kwargs
            }
        })
        
        response = result["generator"]["replies"][0]
        
        if self.verbose:
            print(f"[{self.role}] Task completato.")
        
        return response

In [ ]:
class HaystackTask:
    """Classe per rappresentare un task nel sistema Haystack"""
    
    def __init__(self, description: str, agent: HaystackAgent, expected_output: str = "", 
                 context_tasks: List['HaystackTask'] = None, output_file: str = ""):
        self.description = description
        self.agent = agent
        self.expected_output = expected_output
        self.context_tasks = context_tasks or []
        self.output_file = output_file
        self.output = None
        self.executed = False
    
    def execute(self, **kwargs) -> str:
        """Esegue il task"""
        if self.executed:
            return self.output
        
        # Raccoglie i risultati dai task di contesto
        context_data = ""
        if self.context_tasks:
            context_outputs = []
            for ctx_task in self.context_tasks:
                if not ctx_task.executed:
                    ctx_task.execute(**kwargs)
                context_outputs.append(f"Output da {ctx_task.agent.role}: {ctx_task.output}")
            context_data = "\n\n".join(context_outputs)
        
        # Esegue il task
        self.output = self.agent.execute_task(
            task_description=self.description,
            expected_output=self.expected_output,
            context_data=context_data,
            **kwargs
        )
        
        self.executed = True
        return self.output

In [ ]:
def load_agents(path) -> Dict[str, HaystackAgent]:
    """Carica la configurazione degli agenti dal file YAML"""
    with open(path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)
    
    agents = {}
    
    for key, cfg in data.items():
        # Configurazione di default per llm_config
        default_llm_config = {
            "model": "openai/gpt-3.5-turbo",
            "temperature": 0.7
        }
        
        llm_config = cfg.get("llm_config", default_llm_config)
        
        agent = HaystackAgent(
            role=cfg["role"],
            goal=cfg["goal"],
            backstory=cfg["backstory"],
            llm_config=llm_config,
            verbose=cfg.get("verbose", False)
        )
        agents[key] = agent
    
    return agents

def load_tasks(path, agents_dict: Dict[str, HaystackAgent]) -> List[HaystackTask]:
    """Carica la configurazione dei task dal file YAML"""
    with open(path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)

    task_objs = {}
    
    # Prima passata: crea tutti i task
    for key, cfg in data.items():
        agent = agents_dict[cfg["agent"]]
        task = HaystackTask(
            description=cfg["description"],
            expected_output=cfg.get("expected_output", ""),
            agent=agent,
            output_file=cfg.get("output_file", f"outputs/{key}.md")
        )
        task_objs[key] = task

    # Seconda passata: imposta le dipendenze di contesto
    for key, cfg in data.items():
        if "context" in cfg:
            task_objs[key].context_tasks = [task_objs[cid] for cid in cfg["context"]]

    return list(task_objs.values())

In [ ]:
class HaystackCrew:
    """Classe per coordinare l'esecuzione dei task multi-agente"""
    
    def __init__(self, agents: List[HaystackAgent], tasks: List[HaystackTask], verbose: bool = False):
        self.agents = agents
        self.tasks = tasks
        self.verbose = verbose
    
    def kickoff(self, inputs: Dict[str, Any]) -> str:
        """Avvia l'esecuzione di tutti i task"""
        if self.verbose:
            print("Avvio del sistema multi-agente Haystack...")
        
        results = []
        
        for i, task in enumerate(self.tasks):
            if self.verbose:
                print(f"\nEseguendo task {i+1}/{len(self.tasks)}: {task.agent.role}")
            
            result = task.execute(**inputs)
            results.append(result)
            
            if self.verbose:
                print(f"Task {i+1} completato.")
        
        if self.verbose:
            print("\nTutti i task sono stati completati!")
        
        # Restituisce l'output dell'ultimo task
        return results[-1] if results else ""

In [ ]:
# Caricamento degli agenti e dei task
agents = load_agents("agents.yaml")
tasks = load_tasks("tasks.yaml", agents)

print(f"Caricati {len(agents)} agenti e {len(tasks)} task.")
print("\nAgenti disponibili:")
for key, agent in agents.items():
    print(f"- {key}: {agent.role}")

In [ ]:
# Creazione del crew Haystack
crew = HaystackCrew(
    agents=list(agents.values()),
    tasks=tasks,
    verbose=True
)

In [ ]:
# Parametri di input per il sistema
inputs = {
    'project_name': project_name,
    'project_type': project_type,
    'project_requirements': project_requirements
}

print("Parametri del progetto:")
print(f"Nome: {project_name}")
print(f"Tipo: {project_type}")
print(f"Numero di requisiti: {len(project_requirements)}")

In [ ]:
# Esecuzione del sistema multi-agente
print("Avvio dell'esecuzione del sistema multi-agente...\n")

result = crew.kickoff(inputs=inputs)

print("\n=== RISULTATO FINALE ===")
print(result)

In [ ]:
def save_task_outputs(tasks: List[HaystackTask]):
    """Salva gli output di tutti i task nei rispettivi file"""
    print("\nSalvataggio degli output...")
    
    for task in tasks:
        if not task.executed or not task.output:
            continue
            
        filepath = task.output_file
        if not filepath:
            # Se non c'è output_file, genera un nome dal ruolo dell'agente
            name = task.agent.role.lower().replace(" ", "_")
            filepath = f"outputs/{name}_output.md"
        
        # Crea la directory se non esiste
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        # Salva l'output
        with open(filepath, "w", encoding="utf-8") as f:
            # Aggiungi un header con informazioni sul task
            f.write(f"# Output da: {task.agent.role}\n\n")
            f.write(f"**Obiettivo:** {task.agent.goal}\n\n")
            f.write(f"**Task:** {task.description}\n\n")
            f.write("---\n\n")
            f.write(task.output)
        
        print(f"✓ Salvato: {filepath}")

# Salva tutti gli output
save_task_outputs(tasks)
print("\nTutti gli output sono stati salvati nella cartella 'outputs/'.")

In [ ]:
# Riepilogo dell'esecuzione
print("\n=== RIEPILOGO ESECUZIONE ===")
print(f"Progetto: {project_name}")
print(f"Tipo: {project_type}")
print(f"Agenti utilizzati: {len(agents)}")
print(f"Task eseguiti: {len([t for t in tasks if t.executed])}")
print(f"File generati: {len([t for t in tasks if t.executed and t.output])}")

print("\nTask eseguiti:")
for i, task in enumerate(tasks):
    status = "✓" if task.executed else "✗"
    print(f"{status} {i+1}. {task.agent.role}: {task.description[:80]}...")

print("\nSistema multi-agente Haystack completato con successo!")